#### Credentials

In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

#### Instantiation
Instantiate model object and generate chat completions:

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [3]:
ai_msg = llm.invoke('what is computer science')
ai_msg

AIMessage(content="Computer science (CS) is a broad and dynamic field that encompasses the study of computation and information. It's not just about computers themselves, but rather about the underlying principles and theories that govern how information is processed, stored, and communicated.\n\nHere's a breakdown of key aspects of computer science:\n\n**Core Concepts:**\n\n*   **Algorithms:**  A set of well-defined instructions for solving a problem or performing a task.  Computer scientists design, analyze, and optimize algorithms for efficiency and effectiveness.\n*   **Data Structures:**  Ways of organizing and storing data to enable efficient access and manipulation.  Examples include arrays, linked lists, trees, graphs, and hash tables.\n*   **Programming Languages:**  Formal languages used to instruct computers to perform specific tasks.  Examples include Python, Java, C++, JavaScript, and many others.\n*   **Computation:** The process of performing calculations or logical oper

In [9]:
ai_msg.content.strip()

"Computer science (CS) is a broad and dynamic field that encompasses the study of computation and information. It's not just about computers themselves, but rather about the underlying principles and theories that govern how information is processed, stored, and communicated.\n\nHere's a breakdown of key aspects of computer science:\n\n**Core Concepts:**\n\n*   **Algorithms:**  A set of well-defined instructions for solving a problem or performing a task.  Computer scientists design, analyze, and optimize algorithms for efficiency and effectiveness.\n*   **Data Structures:**  Ways of organizing and storing data to enable efficient access and manipulation.  Examples include arrays, linked lists, trees, graphs, and hash tables.\n*   **Programming Languages:**  Formal languages used to instruct computers to perform specific tasks.  Examples include Python, Java, C++, JavaScript, and many others.\n*   **Computation:** The process of performing calculations or logical operations. Computer s

In [4]:
user_request = "Get employees hired after 2020."

prompt = f"""
    Convert the following request into a valid SQL query:
    "{user_request}"
    The output should be only the SQL query without explanations.
    """

test = llm.invoke(prompt)
test

AIMessage(content="```sql\nSELECT * FROM Employees WHERE HireDate > '2020-12-31';\n```", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b058f115-16f5-481a-b5e5-46a392faf41a-0', usage_metadata={'input_tokens': 40, 'output_tokens': 26, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}})

In [30]:
def generate(prompt):
    # response = genai.generate_text(model="gemini-pro", prompt=prompt)
    response = llm.invoke(prompt)
    return response.content.strip()

setup simple database

In [31]:
import sqlite3
# Setup: Create a sample in-memory database
def setup_database():
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name TEXT,
            hire_date TEXT
        )
    """)
    cursor.executemany("INSERT INTO employees (name, hire_date) VALUES (?, ?)", [
        ("Alice", "2021-05-10"),
        ("Bob", "2019-08-15"),
        ("Charlie", "2022-11-30")
    ])
    conn.commit()
    return conn

agents

In [32]:
import re

def clean_sql_output(sql_response):
    """Removes markdown formatting and unnecessary text from Gemini's output."""
    return re.sub(r"```sql|```", "", sql_response).strip()

# Function to generate SQL query
def generate_sql_query(user_request):
    prompt = f"""
    Given the following database schema: 
    Table: employees (columns: id INTEGER, name TEXT, hire_date TEXT)
    Convert the following request into a valid SQL query:
    "{user_request}"
    The output should be only the SQL query without explanations.
    """
    sql_query = generate(prompt)
    return clean_sql_output(sql_query)

# Function to execute SQL query
def execute_sql_query(query, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()
    except Exception as e:
        return f"Error: {str(e)}"

# Function to verify if result matches intent
def verify_output(user_request, query, result):
    prompt = f"""
    User requested: "{user_request}"
    Generated SQL: "{query}"
    SQL result: "{result}"
    
    Does this output correctly satisfy the user request? Answer with 'Yes' if it does; otherwise, explain why and suggest a corrected SQL query.
    """
    return generate(prompt)

main agent

In [35]:
# Main agent function with retries
def agentic_sql_executor(user_request, conn, max_retries=3):
    for attempt in range(max_retries):
        # Step 1: Generate SQL
        query = generate_sql_query(user_request)
        
        # Step 2: Execute SQL
        result = execute_sql_query(query, conn)
        
        # Step 3: Validate result
        verification = verify_output(user_request, query, result)
        
        if verification.strip().lower().startswith("yes"):
            # q= query
            # r= result
            print(f"Retry {attempt+1}: Fixing based on feedback: {verification}")

            return query, result  # Success!
        else:
            print(f"Retry {attempt+1}: Fixing based on feedback: {verification}")
            # return None, "Failed to generate a valid query."  # Ensure two values are returned
    
    
    # return q, r  # Ensure two values are returned
    return None, "Failed to generate a valid query."  # Ensure two values are returned

test

In [38]:
# Example Usage
if __name__ == "__main__":
    conn = setup_database()
    user_request = "Get employees hired after 2020."
    # user_request = "Get balanced income of each employee after 2020."
    query, result = agentic_sql_executor(user_request, conn)
    print("Final Query:", query)
    print("Final Result:", result)

Retry 1: Fixing based on feedback: Yes
Final Query: SELECT * FROM employees WHERE hire_date > '2020-12-31';
Final Result: [(1, 'Alice', '2021-05-10'), (3, 'Charlie', '2022-11-30')]


In [29]:
tq = "SELECT name FROM sqlite_master WHERE type='table';"
cursor = conn.cursor()
cursor.execute(tq)
test = cursor.fetchall()
test

[('employees',)]